In [1]:
%config Completer.use_jedi = False

In [20]:
!pip install selenium webdriver-manager

In [2]:
import requests as req
import bs4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

import time
import random
import math
import json
import abc

print("Hello World")

Hello World


In [3]:
CHROM_DRIVE_PATH = "/workspace/chromedriver/133.0.6943.142/chromedriver.exe"

In [5]:
service = Service(CHROM_DRIVE_PATH)
driver = webdriver.Chrome(service=service)

actions = ActionChains(driver)

In [8]:
driver.get(BASE_URL)

In [6]:
class Stock(metaclass=abc.ABCMeta):    
    @abc.abstractmethod
    def get_categoty_list(self):
        pass
    
    @abc.abstractmethod
    def get_stock_list(self):
        pass
    
    @abc.abstractmethod
    def get_page_rate(self):
        pass

In [7]:
class UsaStocks(Stock):
    def __init__(self):
        self.BASE_URL = "https://m.stock.naver.com/worldstock/home/USA/industry"
    
    def get_categoty_list(self):
        driver.get(stock.BASE_URL)
        time.sleep(1 + random.random())

        res = driver.page_source
        soup = bs4.BeautifulSoup(res)
        list_items = soup.select("ul.StockCategory_list__IuLu4 > li")
        return list(map(lambda x : x.text, list_items))

    def get_stock_list(self):
        def formatting(row):
            col_list = row.select("td")

            code = col_list[0].select("span")[0].text 

            company_name = col_list[0].select("span")[1].text

            # 현재가, 증감액
            current_price = float(col_list[1].select("span")[0].text.replace(",", ""))
            change_price = float(col_list[1].select("span")[1].select("span")[0].text.replace(",", ""))

            # 증감 비율
            change_rate = float(col_list[2].text[:-1])

            # 시가 총액, 거래량
            market_cap = int(col_list[3].text.replace(",", "") + "000")
            trade_value = col_list[4].text.replace(",", "")
            if(trade_value == "-"):
                trade_value = 0
            trade_value = int(trade_value)

            return {
                "code": code,
                "회사명": company_name,
                "현재가": current_price,
                "증감액": change_price,
                "증감비율": change_rate,
                "시가총액": market_cap,
                "거래량": trade_value,
            }

        res = driver.page_source
        soup = bs4.BeautifulSoup(res)

        return list(map(formatting, soup.select("tr.TableComm_tr__QEe9V")))

    def get_page_rate(self):
        res = driver.page_source
        soup = bs4.BeautifulSoup(res)
        rate_num = soup.select(".VMore_link__Tsoh9 span")[1].text.split(" / ")
        return tuple(map(int, rate_num))
    
    def search(self):
        res = []
        categoty_list = self.get_categoty_list()
        category_count = len(categoty_list)

        for n, category in enumerate(categoty_list):
            progress_bar(n, category_count, category)
            list_itme = driver.find_element(by=By.XPATH, value=f'//*[@id="stockContentWrapper"]/div[8]/ul/li[{n + 1}]')
            driver.execute_script(
                "arguments[0].scrollIntoView({block: 'center', inline: 'nearest'});", list_itme
            )
            list_itme.click()

            time.sleep(random.random())

            rate = 0, 1

            while(True):
                try:
                    reate = get_page_rate()
                    break
                except:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(1 + random.random())


            for i in range(math.ceil(rate[1] / max(rate[0], 1))):
                try:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                    more_button = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[1]/div[2]/div/div/div[10]/div/a')
                    actions.move_to_element(more_button).perform()
                    more_button.click()

                    time.sleep(0.1 + random.random())
                except:
                    time.sleep(2 + random.random())

            stock_list = get_stock_list()

            res.append({
                "카테코리" : category,
                "주식 목록" : stock_list,
            })

            driver.execute_script("window.scrollTo(0, 0);")

            prev_button = driver.find_element(by=By.XPATH, value='//*[@id="stockContentWrapper"]/div[8]/div/a')
            prev_button.click()

            time.sleep(1.5 + random.random())
        #     break
            if(n == 4):
                break
                
        return res

In [18]:
class KoreaStocks(Stock):
    def __init__(self):
        self.BASE_URL = "https://m.stock.naver.com/domestic/industry"
    
    def get_categoty_list(self):
        def formatting(tag):
            return tag.select("span")[0].text
        
        driver.get(self.BASE_URL)
        time.sleep(1 + random.random())
        res = driver.page_source
        soup = bs4.BeautifulSoup(res)
        list_items = soup.select("ul.GroupList_list__ICTC8 > li")
        return list(map(formatting, list_items))

    def get_stock_list(self):
        def formatting(row):
            col_list = row.select("td")

            # 회사명, 코드
            company_name = col_list[0].select("span")[0].text 
            code = col_list[0].select("span")[1].text

            # 현재가, 증감액
            div = col_list[1].select_one("div")
            try:
                change_price = div.select_one(".VGap_iconArrow__4xqtu").text
                change_price = int(change_price.replace(",", ""))
            except AttributeError:
                change_price = 0

            div.decompose()


            current_price = col_list[1].text
            current_price = int(current_price.replace(",", ""))


            # 증감 비율
            change_rate = float(col_list[2].text[:-1])
            change_rate

            # 시가 총액, 거래량
            market_cap = int(col_list[3].text.replace(",", "") + "00000000")
            trade_value = col_list[4].text.replace(",", "")
            if(trade_value == "-"):
                trade_value = 0
            trade_value = int(trade_value)

            return {
                "code": code,
                "회사명": company_name,
                "현재가": current_price,
                "증감액": change_price,
                "증감비율": change_rate,
                "시가총액": market_cap,
                "거래량": trade_value,
            }


        res = driver.page_source
        soup = bs4.BeautifulSoup(res)

        return list(map(formatting, soup.select("tbody tr.CommTable_tr__6yY3y")))
        

    def get_page_rate(self):
        res = driver.page_source
        soup = bs4.BeautifulSoup(res)
        rate_num = soup.select(".VMore_link__Tsoh9 span")[1].text.split(" / ")
        return tuple(map(int, rate_num))

In [9]:
def progress_bar(current, total, category, length=20):
    ratio = current / total  # 진행률 계산
    filled_blocks = int(length * ratio)  # ■ 개수
    empty_blocks = length - filled_blocks  # □ 개수
    bar = "■" * filled_blocks + "□" * empty_blocks
    percent = round(ratio * 100, 2)  # 퍼센트 표시 (소수점 2자리)
    print(f"[{bar}] {percent}% ({current}/{total}, {category})", end="\r")

In [20]:
stock = KoreaStocks()
categoty_list = stock.get_categoty_list()
categoty_list

['인터넷과카탈로그소매',
 '다각화된소비자서비스',
 '운송인프라',
 '창업투자',
 '가정용품',
 '카드',
 '기타금융',
 '부동산',
 '무역회사와판매업체',
 '식품과기본식료품소매',
 '종이와목재',
 '생명과학도구및서비스',
 '무선통신서비스',
 '출판',
 '컴퓨터와주변기기',
 '건축제품',
 '방송과엔터테인먼트',
 '사무용전자제품',
 '도로와철도운송',
 '기타']

In [23]:
res = []

category_count = len(categoty_list)

for n, category in enumerate(categoty_list):
    progress_bar(n + 1, category_count, category)
    list_itme = driver.find_element(by=By.XPATH, value=f'//*[@id="stockContentWrapper"]/div[5]/ul/li[{n + 1}]')
    driver.execute_script(
        "arguments[0].scrollIntoView({block: 'center', inline: 'nearest'});", list_itme
    )
    list_itme.click()
    
    time.sleep(random.random())
    
    rate = 0, 1
    
    while(True):
        try:
            reate = stock.get_page_rate()
            break
        except:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1 + random.random())
        
    
    for i in range(math.ceil(rate[1] / max(rate[0], 1))):
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.1 + random.random())
        except:
            time.sleep(2 + random.random())
            
    stock_list = stock.get_stock_list()
    
    res.append({
        "카테코리" : category,
        "주식 목록" : stock_list,
    })
    
    driver.execute_script("window.scrollTo(0, 0);")
    
    prev_button = driver.find_element(by=By.XPATH, value='//*[@id="stockContentWrapper"]/div[5]/div/a')
    prev_button.click()
    
    time.sleep(1.5 + random.random())
#     break
#     if(n == 2):
#         break

KeyboardInterrupt: 

In [24]:
res

[{'카테코리': '인터넷과카탈로그소매',
  '주식 목록': [{'code': '257720',
    '회사명': '실리콘투',
    '현재가': 26650,
    '증감액': 850,
    '증감비율': 3.29,
    '시가총액': 1630200000000,
    '거래량': 2939727},
   {'code': '069920',
    '회사명': '엑시온그룹',
    '현재가': 818,
    '증감액': 15,
    '증감비율': 1.87,
    '시가총액': 28800000000,
    '거래량': 47762},
   {'code': '362320',
    '회사명': '청담글로벌',
    '현재가': 6180,
    '증감액': 0,
    '증감비율': 0.0,
    '시가총액': 127000000000,
    '거래량': 2296541},
   {'code': '057050',
    '회사명': '현대홈쇼핑',
    '현재가': 52100,
    '증감액': 400,
    '증감비율': -0.76,
    '시가총액': 625200000000,
    '거래량': 19071},
   {'code': '035080',
    '회사명': '그래디언트',
    '현재가': 13280,
    '증감액': 160,
    '증감비율': -1.19,
    '시가총액': 181600000000,
    '거래량': 39365},
   {'code': '475460',
    '회사명': '미트박스',
    '현재가': 9780,
    '증감액': 120,
    '증감비율': -1.21,
    '시가총액': 54600000000,
    '거래량': 42331},
   {'code': '036030',
    '회사명': '케이티알파',
    '현재가': 3695,
    '증감액': 100,
    '증감비율': -2.64,
    '시가총액': 181100000000,
    '거래량': 60085}

In [279]:
with open("stock_data.json", "w", encoding="utf-8") as file:
    json.dump(res, file, ensure_ascii=False, indent=4)  